In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

Mounted at /content/gdrive


In [0]:
import zipfile
z = zipfile.ZipFile('VOC2012.zip', 'r')
z.extractall(path = r"/content")
z.close()

In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

import random

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


from os.path import join as pjoin

import tensorflow as tf
from tensorflow import keras

import time


In [0]:
def LoadImage(name, path_seg = "/content/VOC2012/SegmentationClass",
              path_img="/content/VOC2012/JPEGImages", size=(256, 256)):
    img = Image.open(path_img+"/" + name + ".jpg")
    seg = Image.open(path_seg+"/" + name + ".png").convert("RGB")
    img = np.array(img)
    seg = np.array(seg)
    
    
    img = Image.fromarray(img).resize(size)
    seg = Image.fromarray(seg).resize(size)
    
    
    img = np.array(img)
    seg = np.array(seg)
    return img/255, seg

In [0]:
f = open("/content/VOC2012/ImageSets/Segmentation/train.txt")
lines = f.readlines()
train_list = []
for line in lines:
  line = line.rstrip()
  train_list.append(line) 
f = open("/content/VOC2012/ImageSets/Segmentation/val.txt")
lines = f.readlines()
val_list = []
for line in lines:
  line = line.rstrip()
  val_list.append(line) 


In [0]:
def get_palette():
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128]
            ]
        )

In [0]:
#n_classes = 21
#def rgb_to_onehot(seg):
#    cls = np.zeros((seg.shape[0], seg.shape[1], n_classes))
#    for i in range(128):
#         for j in range(128):
#             for k in range(21):
#               if np.all(seg[i][j] == get_pascal_labels()[k]):
#                  cls[i][j][k] = 1.0
           
#    return cls


In [0]:
n_classes = 21
def one_hot_encoder(seg):
    label_mask = np.zeros((seg.shape[0], seg.shape[1]), dtype=int)
    for ii, label in enumerate(get_palette()):
        label_mask[np.where(np.all(seg == label, axis=-1))[:2]] = ii
    ohe = np.zeros((seg.shape[0], seg.shape[1],n_classes))

    for i in range(1, n_classes):
        m = np.copy(label_mask)
        #m[:,:] = 1
        m[m!=i] = 0
        m[m!=0] = 1
        #m[m==i] = 1
      
        ohe[:,:,i]=m
    m = np.copy(label_mask)
    m[m==0] = -1
    m[m!=-1] = 0
    m[m==-1] = 1
    ohe[:,:,0]=m
    
    return ohe


In [0]:
def Generate_train(batch_size=8):
    iterations = int(len(train_list)/batch_size)
    while True:
        #random.shuffle(train_list)
        for i in range(iterations):
            imgs=[]
            segs=[]
            for j in range(batch_size):
                img, seg = LoadImage(train_list[i * batch_size + j])
                seg = one_hot_encoder(seg)
                imgs.append(img)
                segs.append(seg)
            yield np.array(imgs), np.array(segs)          
train_gen = Generate_train()

In [0]:
def Generate_validation(batch_size=12):
    iterations = int(len(val_list)/batch_size)
    while True:
        #random.shuffle(val_list)
        for i in range(iterations):
            imgs=[]
            segs=[]
            for j in range(batch_size):
                img, seg = LoadImage(val_list[i * batch_size + j])
                seg = one_hot_encoder(seg)
                imgs.append(img)
                segs.append(seg)
            yield np.array(imgs), np.array(segs)          
val_gen = Generate_validation()

In [0]:
inp = Input(shape=(256, 256, 3))

x1 = BatchNormalization()(inp)
x1 = Conv2D(32, (3,3), activation="relu", padding="same")(x1)
x1 = Conv2D(32, (3,3), activation="relu", padding="same")(x1)
p1 = MaxPooling2D()(x1)

#x2 = BatchNormalization()(x1)
x2 = Conv2D(64, (3,3), activation="relu", padding="same")(p1)
x2 = Conv2D(64, (3,3), activation="relu", padding="same")(x2)
p2 = MaxPooling2D()(x2)

#x3 = BatchNormalization()(x2)
x3 = Conv2D(128, (3,3), activation="relu", padding="same")(p2)
x3 = Conv2D(128, (3,3), activation="relu", padding="same")(x3)
p3 = MaxPooling2D()(x3)

#x4 = BatchNormalization()(x3)
x4 = Conv2D(256, (3,3), activation="relu", padding="same")(p3)
x4 = Conv2D(256, (3,3), activation="relu", padding="same")(x4)

#x5 = BatchNormalization()(x4)
x5 = UpSampling2D()(x4)
x5 = concatenate([x3, x5])
x5 = Conv2D(128, (3,3), activation="relu", padding="same")(x5)
x5 = Conv2D(128, (3,3), activation="relu", padding="same")(x5)

#x6 = BatchNormalization()(x5)
x6 = UpSampling2D()(x5)
x6 = concatenate([x2, x6])
x6 = Conv2D(64, (3,3), activation="relu", padding="same")(x6)
x6 = Conv2D(64, (3,3), activation="relu", padding="same")(x6)

#x7 = BatchNormalization()(x6)
x7 = UpSampling2D()(x6)
x7 = concatenate([x1, x7])
x7 = Conv2D(32, (3,3), activation="relu", padding="same")(x7)
x7 = Conv2D(32, (3,3), activation="relu", padding="same")(x7)

out = Conv2D(21, (3,3), activation="softmax", padding="same")(x7)

model = Model(inp, out)



In [0]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd,
             loss="categorical_crossentropy",
             metrics=[tf.keras.metrics.MeanIoU(num_classes=21), "accuracy"])
model.summary()

In [0]:
#opt = Adamax(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#model.compile(optimizer=opt,
#             loss="categorical_crossentropy",
#             metrics=['accuracy'])
#def scheduler(epoch):
#    if epoch % 10 == 0 and epoch != 0:
#        lr = K.get_value(model.optimizer.lr)
#        K.set_value(model.optimizer.lr, lr * 0.1)
#        print("lr changed to {}".format(lr * 0.1))
#    return K.get_value(model.optimizer.lr)

# def poly_decay(epoch):
#     # initialize the maximum number of epochs, base learning rate,
#     # and power of the polynomial
#     maxEpochs = epochs
#     baseLR = learning_rate
#     power = 1.0
# 
#     # compute the new learning rate based on polynomial decay
#     alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
# 
#     return alpha
# 
# 
# poly_reduce_lr = LearningRateScheduler(poly_decay)
 
#reduce_lr = LearningRateScheduler(scheduler)

In [0]:
model.run_eagerly = True

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()


In [0]:
result = model.fit_generator(train_gen, steps_per_epoch = float(len(train_list)/12), validation_data = val_gen, 
                              validation_steps = float(len(val_list)/12), epochs = 50, callbacks=[time_callback], verbose = 1)

Epoch 1/50
  9/122 [=>............................] - ETA: 56s - loss: 2.4252 - mean_io_u_1: 0.4762 - accuracy: 0.5531

KeyboardInterrupt: ignored

In [0]:
N = len(result.history['loss'])

plt.style.use("ggplot")
fig = plt.figure(figsize=(20,8))

fig.add_subplot(1,2,1)
plt.title("Loss")
plt.plot(np.arange(0, N), result.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), result.history["val_loss"], label="val_loss")
plt.ylim(0, 3)

fig.add_subplot(1,2,2)
plt.title("Accuracy")
plt.plot(np.arange(0, N), result.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), result.history["val_accuracy"], label="val_accuracy")
plt.ylim(0, 1)

plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

NameError: ignored

In [0]:
result.history
print(time_callback.times)

[4145.8036279678345, 4274.622194766998]
